# Finding hyperplanes on the multidimensional grids

## Table of Content <a name="TOC"></a>

1. [General setups](#setups)
2. [Mapping points on multidimensional grids ](#mapping)
3. [Finding the hyperplanes](#hyperplane)

   3.1. [Method 1](#method1)

   3.2. [Method 2](#method2)

   3.3. [Method 3](#method3)

   3.4. [Method 4](#method4)

### A. Learning objectives

- to map sequential numbers of the grid points to the multi-dimensional index and vice versa
- to find all the grid points that belong to a certain hyperplane

### B. Use cases

- [Finding a hyperplane of a multidimensional grid](#hyperplane)
- [Mapping integers and vectors of ints on regular grids](#mapping)

### C. Functions

- `liblibra::libdyn::libwfcgrid`    
  - [`compute_imapping`](#compute_imapping-1)
  - [`compute_hyperplane`](#compute_hyperplane-1)
  - [`compute_mapping`](#compute_mapping-1)


### D. Classes and class members


## 1. General setups 
<a name="setups"></a>[Back to TOC](#TOC)

First, import all the necessary libraries:
* liblibra_core - for general data types from Libra

The output of the cell below will throw a bunch of warnings, but this is not a problem nothing really serios. So just disregard them.

In [1]:
import os
import sys
import math
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
from libra_py import data_outs


/mnt/d/WORK/SOFTWARE/Conda/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/mnt/d/WORK/SOFTWARE/Conda/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/mnt/d/WORK/SOFTWARE/Conda/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<flo

## 2. Mapping points on multidimensional grids 
<a name="mapping"></a>[Back to TOC](#TOC)


Imagine a 3D grid with:
 * 3 points in the 1-st dimension
 * 2 points in the 2-nd dimension  
 * 4 points in the 3-rd dimension

So there are 3 x 2 x 4 = 24 points 
 
However, we can still store all of them in 1D array, which is more efficient way. However, to refer to the points, we need a function that does the mapping.

This example demonstrates the functions:

`vector<vector<int> > compute_mapping(vector<vector<int> >& inp, vector<int>& npts)`

`int compute_imapping(vector<int>& inp, vector<int>& npts)`

defined in:   dyn/wfcgrid/Grid_functions.h


The mapping formula for grids of different dimensions is given by:

    1D: i = i_0 
    2D: i = i_0*n_1 + i_1
    3D: i = i_0*n_1*n_2 + i_1*n_2 + i_2 

and so on.

Here is how such a mapping can be implemented in Python:
<a name="compute_mapping-1"></a>

In [2]:
def compute_imapping_py(inp, npts):
    """
    Maps vector to an indeger according to the grid dimensions
    """

    ndof = len(npts)

    i = inp[0];
    for dof in range(1, ndof):   
        i = i * npts[dof] + inp[dof]

    return i

Here, the first argument is the vector representing the point of interest, the second argument indicates how many points there are for each dimension:

In [3]:
pt_indx = compute_imapping_py([5], [10])
print(pt_indx)

pt_indx = compute_imapping_py([0,0], [3,3])
print(pt_indx)

pt_indx = compute_imapping_py([0,2], [3,3])
print(pt_indx)

pt_indx = compute_imapping_py([0, 2, 2], [3,3,3])
print(pt_indx)

5
0
2
8


This can also be done with the built-in Libra function `compute_imapping`: <a name="compute_imapping-1"></a>

In [4]:
pt_indx = compute_imapping(Py2Cpp_int([5]), Py2Cpp_int([10]) )
print(pt_indx)

pt_indx = compute_imapping(Py2Cpp_int([0,0]), Py2Cpp_int([3,3]) )
print(pt_indx)

pt_indx = compute_imapping(Py2Cpp_int([0,2]), Py2Cpp_int([3,3]) )
print(pt_indx)

pt_indx = compute_imapping(Py2Cpp_int([0, 2, 2]) , Py2Cpp_int([3,3,3]) )
print(pt_indx)

5
0
2
8


Now, if we want to do the inverse mapping, we can use the following Python function:

In [5]:
def compute_mapping_py(indx, npts):
    """
    Maps an integer to a vector according to the grid dimensions
    """

    ndof = len(npts)
    sizes = []
    sz = 1
    for i in range(ndof):
        sizes.append(sz)
        sz = sz * npts[ndof-1-i]

    res = []
    _indx = indx
    for i in range(ndof):
        rem = _indx % sizes[ndof-1-i]
        ni = (_indx - rem) / sizes[ndof-1-i]
        _indx = _indx - ni*sizes[ndof-1-i]

        res.append(int(ni))
 
    return res


Here, again, the second argument describes the grid structure. However, the first argument is the index of a point of interest. The function should return the arguments used as the inputs in the examples above: <a name="compute_mapping-1"></a>

In [6]:
res = compute_mapping_py(5, [10])
print(res)

res = compute_mapping_py(0, [3,3])
print(res)

res = compute_mapping_py(2, [3,3])
print(res)

res = compute_mapping_py(8, [3,3,3])
print(res)

[5]
[0, 0]
[0, 2]
[0, 2, 2]


Analogous Python-exposed C++ function is `compute_mapping`:

In [7]:
res = compute_mapping(5, Py2Cpp_int([10]) )
print(Cpp2Py(res) )

res = compute_mapping(0, Py2Cpp_int([3,3]) )
print(Cpp2Py(res) )

res = compute_mapping(2, Py2Cpp_int([3,3]) )
print(Cpp2Py(res) )

res = compute_mapping(8, Py2Cpp_int([3,3,3]) )
print(Cpp2Py(res) )

[5]
[0, 0]
[0, 2]
[0, 2, 2]


Note that there is one more `compute_mapping` function in Libra. It would compute the list of all points available in a given grid:

In [8]:
inp = intList2()
npts = Py2Cpp_int([3,3,3])

res = compute_mapping(inp, npts);

print("The number of points = ", len(res) )
print("The number of dimensions = ", len(res[0]) )

for cnt, i in enumerate(res):
    print("point # ", cnt, Cpp2Py(i) )        

The number of points =  27
The number of dimensions =  3
point #  0 [0, 0, 0]
point #  1 [0, 0, 1]
point #  2 [0, 0, 2]
point #  3 [0, 1, 0]
point #  4 [0, 1, 1]
point #  5 [0, 1, 2]
point #  6 [0, 2, 0]
point #  7 [0, 2, 1]
point #  8 [0, 2, 2]
point #  9 [1, 0, 0]
point #  10 [1, 0, 1]
point #  11 [1, 0, 2]
point #  12 [1, 1, 0]
point #  13 [1, 1, 1]
point #  14 [1, 1, 2]
point #  15 [1, 2, 0]
point #  16 [1, 2, 1]
point #  17 [1, 2, 2]
point #  18 [2, 0, 0]
point #  19 [2, 0, 1]
point #  20 [2, 0, 2]
point #  21 [2, 1, 0]
point #  22 [2, 1, 1]
point #  23 [2, 1, 2]
point #  24 [2, 2, 0]
point #  25 [2, 2, 1]
point #  26 [2, 2, 2]


## 3. Finding the hyperplane
<a name="hyperplane"></a>[Back to TOC](#TOC)

Now, we are going to solve the following problem: given a certain regular multi-dimensional grid with points $(i_0, i_1, i_2, ..., i_{N-1})$, enumerated sequentially (by some index $I$) according to the scheme shown above, find the indices ${I...}$ of all points $(i_0, i_1, ... i_d = X, ... i_{N-1})$ for which the value of the index for a given dimension $d$ is a constant value. We refer to such a subset of points as the hyperplane.


### 3.1. Method 1
<a name="method1"></a>[Back to TOC](#TOC)

In this approach, we first going to systematically create all the vectors, but paying attention that the index of the d-th component (`idim_const` parameter below) takes only one possible (pre-defined) value of `ipt_const`.

First, we define a function that would generate vectors of length `n` from vectors of length `n-1` obeying such the constraints.

In [9]:
def extend_vectors(idim, inpts, npts_at_idof, idim_const, ipt_const ):
    """
    idim (int) - index of the dimension on which we are working on currently
    inpts (list of int lists) - set of incomplete vectors (up to given dimension) 
             representing the grid points
    npts_at_idof (int) - how many points are for a given i
    idim_const (int) - for this dimension we only do 1 fixed point
    ipt_const (int) - is the index of that point along the idim_const direction
    """

    res = []
    if len(inpts)>0:
        for inpt in inpts:
            x = list(inpt)

            if idim == idim_const:
                a = list(x)
                a.append(ipt_const)
                res.append(a)
            else:
                for i in range(npts_at_idof):
                    a = list(x)
                    a.append(i)
                    res.append(a)
    else:
        if idim == idim_const:
            a = [ipt_const]
            res.append(a)
        else:
            for i in range(npts_at_idof):
                a = [i]
                res.append(a)

    return res
        

Now, we simply need to call such a function several times (which is the dimensionality of the grid), starting from an empty list and using the output of the function on a previous iteration as its input on the next. The function would use the generated vectors to determine their indices in the multidimensional grid using the function `compute_imapping_py` defined above. Of course, one could adapt it to work with the Libra's function `compute_imapping` discussed above.

In [10]:
def compute_hyperplane_py(npts, idim_const, ipt_const):
    """
    grid - list of ints with the number of points in each directions

    """

    ndof = len(npts)

    res = [ ]
    for idof in range(ndof):
        res = extend_vectors(idof, res, npts[idof], idim_const, ipt_const)

    indxs = []
    for vec in res:
        indx = compute_imapping_py(vec, npts)
        indxs.append(indx)

    return res, indxs

Now let's see how it works

In [11]:
print("===============")
res, indxs = compute_hyperplane_py([3,3], 0, 0)
print(res, indxs)

print("===============")
res, indxs = compute_hyperplane_py([3,3], 1, 0)
print(res, indxs)

print("===============")
res, indxs = compute_hyperplane_py([3,3,3], 1, 1)
print(res, indxs)


[[0, 0], [0, 1], [0, 2]] [0, 1, 2]
[[0, 0], [1, 0], [2, 0]] [0, 3, 6]
[[0, 1, 0], [0, 1, 1], [0, 1, 2], [1, 1, 0], [1, 1, 1], [1, 1, 2], [2, 1, 0], [2, 1, 1], [2, 1, 2]] [3, 4, 5, 12, 13, 14, 21, 22, 23]


### Exercise 1.

Adapt the function `compute_hyperplane_py` to work with the Libra's function `compute_imapping` instead of the `compute_imapping_py` function.

### 3.2. Method 2
<a name="method2"></a>[Back to TOC](#TOC)

Of course, generating the whole list of all possible vector points may be an expensive procedure, which would also scale pooly for higher dimensions and many points. Thus, we are interested in to return only the indices of the points belonging to the hyperplane, without actually giving their vector representation.

To do this, we can simply decompose the integer index of every point to the vector of ints, using the `compute_mapping_py` (or `compute_mapping`) functions and then check whether the needed component is what it is supposed to be

In [12]:
def compute_hyperplane_py_fast(npts, idim_const, ipt_const):
    
    tot_npts = 1
    for npt in npts:
        tot_npts = tot_npts * npt

    res = []
    for i in range(tot_npts):
        vec = compute_mapping_py(i, npts)
        if vec[idim_const] == ipt_const:
            res.append(i)

    return res


Applied to the examples above, it yields:

In [13]:
indxs2 = compute_hyperplane_py_fast([3,3], 0, 0)
print(indxs2)

indxs2 = compute_hyperplane_py_fast([3,3], 1, 0)
print(indxs2)

indxs2 = compute_hyperplane_py_fast([3,3,3], 1, 1)
print(indxs2)


[0, 1, 2]
[0, 3, 6]
[3, 4, 5, 12, 13, 14, 21, 22, 23]


### Exercise 2.

Adapt the function `compute_hyperplane_py_fast` to work with the Libra's function `compute_mapping` instead of the `compute_mapping_py` function.

### 3.3. Method 3
<a name="method3"></a>[Back to TOC](#TOC)

Finally, we can call the Libra's function `compute_hyperplane` which works exactly as `compute_hyperplane_py_fast` but with all internals hidden and inmplemented in C++ code. The signature is also slightly different.

    compute_hyperplane(vector<int>& npts, int idim_const, int ipt_const)
    

So here is how it works: <a name="compute_hyperplane-1"></a>

In [14]:
res = compute_hyperplane(Py2Cpp_int([3,3]), 0, 0)
print( Cpp2Py(res) )

res = compute_hyperplane(Py2Cpp_int([3,3]), 1, 0)
print( Cpp2Py(res) )

res = compute_hyperplane(Py2Cpp_int([3,3,3]), 1, 1)
print( Cpp2Py(res) )

[0, 1, 2]
[0, 3, 6]
[3, 4, 5, 12, 13, 14, 21, 22, 23]


### 3.4. Finding the hyperplane
<a name="method4"></a>[Back to TOC](#TOC)

This an alternative approach developed by **r. Luke Ali** during the CyberTraining summer school/workshop in July 2022. It is qualitatively different from all the above examples and is quite interesting. It relies on a building a set of expressions (equations) for evaluating the indices of the points belonging to the hyperplane. Then one calls another function to evaluate all these expressions and to determine the required indices. 

In [15]:
def build_equation(dimensions, slice):
    '''
    slice is in the format [axis, point] and represents the hyperplane you want to look at. 
    
    This format makes things a few lines simpler.    
    
    '''

    # Create initial structure of terms
    terms = ''

    # Builds an expression of the form N[x]*N[x-1]*N[x-2]*...*N[1]* where  x = dimensions - 1
    # Notice the error above with * at the end
    for i in range(dimensions - 1, 0, -1):

        terms += 'N[' + str(i) + ']*'

    # Correcting for * error 
    terms = terms[0:-1:] 
    
    
    eq = ''

    for i in range(dimensions):

        # Builds and expression of the form C[0]*N[x]*N[x-1]*N[x-2]*...*N[1]
        eq += 'C[' + str(i) + ']*' + terms + '+'

        # Removes the smallest term in the string eg: N[5]*N[4]*N[3]*N[2] -> N[5]*N[4]*N[3]
        # The reasoning stems from the derivation of the general form
        terms = terms[0:-len('*N[' + str(i+1) + ']')]

        # Builds the final equation that is used to run the calculation
        expression = 'sol.append(' + \
        eq[0:-2].replace('C[' + str(slice[0]) + ']', str(slice[1])) + ')'  

    return expression

def calculate(sol, dimensions, expression, index, C, N, current = 0, key = 0):
    '''
    The main calculation function

    sol is the list that contains all of the points from the output


    N is the grid size in the form of a list. 

    N = [N0, N1, N2, ... , Nn-1]

    Create Coefficient Dictionary in the following way:

    C = {x : 0 for x in range(len(N))}

    '''

    # Used to check whether the last index is the one in question
    # Fixes an index error. 
    if key == index and index == dimensions - 1:

        exec(expression)

    # Prevents iteration over a fixed index. 
    elif key == index:

        # Keeps track of position in the list
        current += 1 

        # Calls itself to iterate over the next index
        calculate(sol, dimensions, expression, index, C, N, current, key + 1)

    # Checks if on last index and calculates the terms. 
    # This is the final step in a recursive branch if index != dimensions - 1
    elif key == dimensions - 1 and current == dimensions - 1: 
        
        for C[key] in range(N[key]): 

            exec(expression)

        # Keeps track of position in the list
        current -= 1

    else:

        # Keeps track of position in the list
        current += 1

        # Recursion to iterate over the next index. 
        for C[key] in range(N[key]):

            calculate(sol, dimensions, expression, index, C, N, current, key + 1)


Let's see how it works:

First, the **1D case**

In [16]:
grid = [10]  # define the grind of 10 points in 1D
coefficients = {x : 0 for x in range(len(grid))}

# Build an expression
dim = 1
slc = [0, 5]  # the grid size [0 - first axis, 5 - 5th point]
expr = build_equation(dim, slc)

# Compute the hyperplane
output = []
calculate(output, dim, expr, slc[0], coefficients, grid)
print(output)


[5]


Now consider 2  **2D cases**, already computed above:

In [17]:
grid = [3,3]  # define the grind of 10 points in 1D
coefficients = {x : 0 for x in range(len(grid))}

# Build an expression
dim = 2
slc = [0, 0]  # the grid size [0 - first axis, 0 - 0th point]
expr = build_equation(dim, slc)

# Compute the hyperplane
output = []
calculate(output, dim, expr, slc[0], coefficients, grid)
print(output)



grid = [3,3]  # define the grind of 10 points in 1D
coefficients = {x : 0 for x in range(len(grid))}

# Build an expression
dim = 2
slc = [1, 0]  # the grid size [1 - second axis, 0 - 0th point]
expr = build_equation(dim, slc)

# Compute the hyperplane
output = []
calculate(output, dim, expr, slc[0], coefficients, grid)
print(output)

[0, 1, 2]
[0, 3, 6]


Finally, a **3D case**

In [18]:
grid = [3,3,3]  # define the grind of 10 points in 1D
coefficients = {x : 0 for x in range(len(grid))}

# Build an expression
dim = 3
slc = [1, 1]  # the grid size [1 - first axis, 1 - 1st point]
expr = build_equation(dim, slc)

# Compute the hyperplane
output = []
calculate(output, dim, expr, slc[0], coefficients, grid)
print(output)

[3, 4, 5, 12, 13, 14, 21, 22, 23]
